# First Solution

![first win](img/first-win-03-16-21.png)

## Future Improvements

* make a queue of squares to check, skipping squares that are already completely accounted for (avoid needless checks)
* there are some scenarios where the bot will have to guess
* make the system grid size agnostic so it's easy to switch between versions
* need a way to verify that the game is complete (instead of infite loop)

In [1]:
def click_square(y, x):
    
    # click the coordinate
    xpath = '//*[@id="' + str(y+1) + '_' + str(x+1) + '"]'
    driver.find_element_by_xpath(xpath).click()
    square_info = driver.find_element_by_xpath(xpath).get_attribute('class')
    
    # collect changed information
    grid[y][x]= int(square_info.split(' ')[1][4:])
    
    # update table information
    full_scan(9, 9)

## Scan Functions

In [2]:
def scan(y, x):
    xpath = '//*[@id="' + str(y+1) + '_' + str(x+1) + '"]'
    square_info = driver.find_element_by_xpath(xpath).get_attribute('class')
    
    if 'open' in square_info:
        grid[y][x] = int(square_info.split(' ')[1][4:])   
        
    if 'flagged' in square_info:
        grid[y][x] = 'f'

In [3]:
def full_scan(h, w):
    
    # scan everything
    for dy in range(h):
        for dx in range(w):
            scan(dy, dx)

## Flag Functions

### Goals

1. Gather Neighbors of an open cell (non blank / non flagged)

In [4]:
from selenium.webdriver import ActionChains

def flag(y, x):
    
    # flag the coordinate
    xpath = '//*[@id="' + str(y+1) + '_' + str(x+1) + '"]'
    flag_element = driver.find_element_by_xpath(xpath)
    
    action = ActionChains(driver)
    action.context_click(flag_element).perform()
    
    # update table information
    full_scan(9, 9)

In [5]:
import itertools

def gather_neighbors(y, x):
    rows = []
    cols = []
    
    score = grid[y][x]
    
    # check boundery constraints
    if (y == 0):
        rows = [0, 1]
    elif (y == 8):
        rows = [7, 8]
    else:
        rows = [y - 1, y, y + 1]
        
    if (x == 0):
        cols = [0, 1]
    elif (x == 8):
        cols = [7, 8]
    else:
        cols = [x - 1, x, x + 1]
    
    neighbor_cords = list(itertools.product(rows, cols))
    neighbor_cords.remove((y, x))
    
    if len(neighbor_cords) != 0:
    
        blanks = []

        for (ny, nx) in neighbor_cords:
            nbr = grid[ny][nx]
            if nbr == '_':
                blanks.append((ny, nx))
            if nbr == 'f':
                score -= 1

        if score == len(blanks):
            for (ny, nx) in blanks:
                flag(ny, nx)

        if score == 0:
            for (ny, nx) in blanks:
                click_square(ny, nx)

In [6]:
def flag_check(y, x):
    if isinstance(grid[y][x], int) and (grid[y][x] > 0):
        gather_neighbors(y, x)

In [7]:
def flag_scan(h, w):
    for y in range(h):
        for x in range(w):
            flag_check(y, x)

## Main Running Function

In [8]:
"""
This is the driver setup
"""

from selenium import webdriver

grid = [['_' for x in range(9)] for y in range(9)]

# setup driver after data structures are created
driver = webdriver.Chrome("./chromedriver.exe")
driver.get("http://minesweeperonline.com/#beginner")

# start in the center
click_square(4, 4)

while (True):
    flag_scan(8, 8)

KeyboardInterrupt: 

In [170]:
driver.close()

## Print Grid

In [10]:
def print_grid(h, w):
    
    line = "-----" * w
    print(line)
    
    for dy in range(h):
        contents = ""
        for dx in range(w):
            contents += "| "
            contents += str(grid[dy][dx])
            contents += " |"
        print(contents)
        print(line)

In [11]:
print_grid(9, 9)

---------------------------------------------
| _ || _ || _ || 3 || f || 1 || 0 || 1 || f |
---------------------------------------------
| _ || _ || _ || f || 2 || 1 || 0 || 1 || 1 |
---------------------------------------------
| _ || _ || _ || 3 || 1 || 0 || 0 || 0 || 0 |
---------------------------------------------
| _ || _ || _ || 1 || 0 || 0 || 0 || 0 || 0 |
---------------------------------------------
| _ || _ || _ || 1 || 0 || 0 || 0 || 0 || 0 |
---------------------------------------------
| _ || _ || _ || 1 || 0 || 0 || 0 || 0 || 0 |
---------------------------------------------
| _ || _ || _ || 2 || 0 || 0 || 0 || 0 || 0 |
---------------------------------------------
| _ || _ || _ || 2 || 0 || 0 || 0 || 0 || 0 |
---------------------------------------------
| _ || _ || _ || 2 || 0 || 0 || 0 || 0 || 0 |
---------------------------------------------
